# hide
title: Walking through the Github archive
tags: data github
enable: plotly

There are addictions to all kinds of things. Currently, i'm addicted to generate timeseries of all kinds of public data. I just finished setting up a [data archive](https://github.com/defgsus/bahn-api-history) for a couple of API responses from the [Deutsche Bahn](https://api.deutschebahn.com) which was a actually just a procrastination tactic to pull myself away from the refactoring of the [office schedule data archive](https://github.com/defgsus/office-schedule-data) which is quite elaborate. More procrastination followed and i started using the [github search](https://github.com/search?o=desc&q=data+archive&s=updated&type=Repositories) for other repositories that collect historic data. There are quite a few interesting ones. Not long it took to stumble across the [github archive](http://www.gharchive.org/) of which i hadn't heared before. 

The github archive is a bunch of compressed, new-line delimited json files, containing all the [events](https://docs.github.com/en/developers/webhooks-and-events/events/github-event-types) of the github events API. Well, maybe not entirely ***all*** requests (there are statements about missing events in the [discussions](https://github.com/igrigorik/gharchive.org/issues)) but enough to make people stumble. 

To get an overview of the pure amount of data, download a single hour of each year since availability (Feb. 2011):

```
$ wget https://data.gharchive.org/20{11..21}-03-01-15.json.gz
```

The numbers are from a horror movie!
```
$ ls -l
    450433 Apr  2  2018 2011-03-01-15.json.gz
   2329840 Apr  2  2018 2012-03-01-15.json.gz
   2648943 Apr  2  2018 2013-03-01-15.json.gz
   3369069 Apr  2  2018 2014-03-01-15.json.gz
   6365599 Apr  2  2018 2015-03-01-15.json.gz
  21268062 Apr  2  2018 2016-03-01-15.json.gz
  25974997 Apr  2  2018 2017-03-01-15.json.gz
  37577449 Apr  2  2018 2018-03-01-15.json.gz
  42593332 Mär  1  2019 2019-03-01-15.json.gz
  35538137 Mär  1  2020 2020-03-01-15.json.gz
 104531294 Mär  1  2021 2021-03-01-15.json.gz
```

In [6]:
# hide
print((450_000 * 24 * 365) // 2**20)
print((21_000_000 * 24 * 365) // 2**20)
print((104_000_000 * 24 * 365) // 2**20)

3759
175437
868835


While the files for year 2011 might fit into 3 Gigabytes, 2016 will have more than a 100 Gb and 2021 is probably more than 500 Gb!

So well, i listened to the [The Changelog episode #144](https://changelog.com/podcast/144) as suggested by gharchive.org and the author [Ilya Grigorik](https://github.com/igrigorik) was obviously struggling with the same problems. The solution at the time was to put everything on Google BigQuery. As Ilya states in the podcast: a *BigQuery* through the whole dataset takes between 1 and 10 seconds, while just reading all the files from disk takes an hour.

Now, i'm getting older and my not-to-do list is growing. These times, whenever i see Google BigQuery or Google Colab Notebook, i skip the link and look for other ways to access what i want. It may just be stubbornness but that's how it is. Google APIs are blocked in my browser and i'm not in the mood to allow them for recreational purposes. 

Let's parse this data ourselves. How difficult can it be?

I have this server with about 1.7Tb of harddisk space left so i'm downloading years 2018 to 2021 (until early November):

```
du -h
175G	./2018
253G	./2019
419G	./2020
443G	./2021
1,3T	.
```

Monstrous! The download took 4 hours or so, the server has a *good* connection. In the meantime i created some [utility code](https://github.com/defgsus/gharchive-stats) that i can run on the server to *bucket* the data for further processing.

As an example let's look at the `WatchEvent` which represents the *starring* of a repository. Here's the first recorded WatchEvent from 2018:

```json
{
  "id": "7044401123",
  "type": "WatchEvent",
  "actor": {
    "id": 1710912,
    "login": "yangwenmai",
    "display_login": "yangwenmai",
    "gravatar_id": "",
    "url": "https://api.github.com/users/yangwenmai",
    "avatar_url": "https://avatars.githubusercontent.com/u/1710912?"
  },
  "repo": {
    "id": 75951828,
    "name": "wainshine/Chinese-Names-Corpus",
    "url": "https://api.github.com/repos/wainshine/Chinese-Names-Corpus"
  },
  "payload": {
    "action": "started"
  },
  "public": true,
  "created_at": "2018-01-01T00:00:02Z"
}
```

The bucketing script produces a CSV like this:

| date                 | user       | repo                                   | action   |   events |
|:---------------------|:-----------|:---------------------------------------|:---------|---------:|
| 2018-01-01T00:00:00Z | yangwenmai | wainshine/Chinese-Names-Corpus         | started  |        1 |

The date is *floored* to the nearest hour and every occurence of an event with the same `date`, `user`, `repo` and `action` are bucketed into a single line, increasing the `events` counter. 

The conversion of **all the WatchEvents in 2018**
 - took eleven hours
 - produced a CSV with
   - 36,585,735 rows
   - and a size of 628,047,892 bytes (compressed with gzip)

Now, that's not a CSV you can just open but at least i can copy it from the server to my local system.  

In [18]:
# hide
import csv
import gzip
import json
from tqdm import tqdm
from pathlib import Path
import pandas as pd
import numpy as np

with gzip.open(Path("~/prog/data/gharchive/2018/2018-01-01-0.json.gz").expanduser()) as fp:
    for line in fp.readlines():
        event = json.loads(line)
        if event["type"] == "WatchEvent":
            print(json.dumps(event, indent=2))
            break

{
  "id": "7044401123",
  "type": "WatchEvent",
  "actor": {
    "id": 1710912,
    "login": "yangwenmai",
    "display_login": "yangwenmai",
    "gravatar_id": "",
    "url": "https://api.github.com/users/yangwenmai",
    "avatar_url": "https://avatars.githubusercontent.com/u/1710912?"
  },
  "repo": {
    "id": 75951828,
    "name": "wainshine/Chinese-Names-Corpus",
    "url": "https://api.github.com/repos/wainshine/Chinese-Names-Corpus"
  },
  "payload": {
    "action": "started"
  },
  "public": true,
  "created_at": "2018-01-01T00:00:02Z"
}


Python's generators are quite useful to walk through a CSV or ndjson file which might not fit into memory in it's entirety. Let's measure the time it takes to find all of *yangwenmai*'s `WatchEvent`s with vanilla python:

In [26]:
import csv
import gzip
import time

# per line iterator through the gzipped csv
def iter_lines():
    with gzip.open("../../../gharchive-stats/server-all/2018/watch_h.csv.gz", "rt") as fp:
        yield from fp.readlines()

rows = []
start_time = time.time()

# filter for user
for row in csv.DictReader(iter_lines()):
    if row["user"] == "yangwenmai":
        rows.append(row)

duration = time.time() - start_time
print(f"took {duration:.2f} seconds")

# convert to pandas
df = pd.DataFrame(rows)
df

took 101.20 seconds


So, a 100 seconds, simply to parse through the CSV, no matter what we are actually looking for. This would probably lead to a query time of 10 minutes for the whole dataset. Far from optimal but it requires no complicated stuff, google accounts or lots of memory. 

For reasons, i quite like [Elastisearch](https://www.elastic.co/elasticsearch/) so i move the CSV file into an elasticsearch index which took 90 minutes and resulted in an index of 4.2 Gb size. Repeating the above query using [elastipy](https://elastipy.readthedocs.io/en/latest/):

In [49]:
from elastipy import Search

response = (Search("gharchive-watch-2018")
 .term("user", "yangwenmai")
 .size(1000)
 .execute()
)
print("took", response["took"], "ms")
pd.DataFrame(response.documents)

took 20 ms


,timestamp,timestamp_hour,timestamp_weekday,user,repo,action,events
0,2018-05-25T23:00:00,23,5 Friday,yangwenmai,developer-learning/learning-kubernetes,started,1
1,2018-05-26T09:00:00,9,6 Saturday,yangwenmai,danistefanovic/build-your-own-x,started,1
2,2018-05-27T03:00:00,3,0 Sunday,yangwenmai,yongman/tidis,started,1
3,2018-05-28T02:00:00,2,1 Monday,yangwenmai,muesli/kmeans,started,1
4,2018-05-28T06:00:00,6,1 Monday,yangwenmai,appoptics/appoptics-apm-go,started,1
...,...,...,...,...,...,...,...
496,2018-11-28T14:00:00,14,3 Wednesday,yangwenmai,mholt/archiver,started,1
497,2018-11-30T02:00:00,2,5 Friday,yangwenmai,henrylee2cn/aster,started,1
498,2018-12-11T14:00:00,14,2 Tuesday,yangwenmai,developer-learning/telegram-bot-go,started,1
499,2018-12-13T00:00:00,0,4 Thursday,yangwenmai,bragfoo/TiPrometheus,started,1


Yes! Praise "Bob!". Who needs the google cloud? 